In [ ]:
from google.colab import drive # Para montar nuestro drive en la consola
import matplotlib.pylab as plt # Para gráficos
import numpy as np # Para manejo de arrays, operaciones matemáticas, etc.
from sklearn.linear_model import LogisticRegression # El método de regresión logística que vamos a usar
import pandas as pd # Para manejo de base de datos
from sklearn.metrics import confusion_matrix # matriz de confusion

drive.mount('/content/drive') # Montamos nuestra unidad de Google Drive
filename = '/content/drive/My Drive/anticoncepcion_indonesia.csv'

d = pd.read_csv(filename) # Levanto los datos
d.head() # Veo las primeras líneas

Mounted at /content/drive


,edad_mujer,educacion_mujer,educacion_hombre,numero_hijos,religion_mujer,mujer_trabaja,ocupacion_hombre,nivel_vida,exposicion_medios,metodo_anticonceptivo
0,35,3,4,3,1,1,2,2,0,2
1,29,4,4,3,1,1,1,4,0,2
2,21,4,4,1,1,1,2,3,0,3
3,26,3,4,3,1,1,3,2,0,3
4,45,3,4,6,1,1,2,4,0,1


In [ ]:
d_filtrado=d.copy()
d_filtrado['Toma Anticonceptivos'] = 0
indice =  d_filtrado['metodo_anticonceptivo'] > 1
d_filtrado.loc[indice, 'Toma Anticonceptivos'] = 1

d_filtrado['Toma Anticonceptivos'].value_counts()
#d_filtrado.head()

1    165
0    131
Name: Toma Anticonceptivos, dtype: int64

Fui limpiando el notebook mientras lo iba haciendo porque la verdad que mucho no sabía lo que hacía, asique voy a describir lo que me acuerdo que probé.

Primero armé la columna que indica si toma o no anticonceptivos.

Después fui probando y cambiando cosas del modelo, cambié los pesos, probé hacer 5 folds, armé el histograma, como en clase, para analizar cuantos features eran convenientes y también para ver si usaba o no potencias de X, de eso saqué la conclusión de que 6 features eran lo mejor (y saque exposición a los medios y ocupación del hombre), y solo tener en cuenta X y X^2, porque sino overfitteaba.

Probé el mapa de color de la clase para definir la constante de regularización del modelo y me pareció que c=16 era como mejor resultaba.


In [ ]:
from sklearn.model_selection import StratifiedKFold

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest

# Definamos la matriz X
campos = ['edad_mujer','educacion_mujer', 'educacion_hombre', 'numero_hijos', 'mujer_trabaja','nivel_vida'] # Lista que contiene las features que definí para usar
X = d_filtrado[campos].values # En este caso no hace falta reshapear, porque ya tiene las dimensiones correctas
y = np.array(d_filtrado['Toma Anticonceptivos'])
X_1 = np.concatenate((X, X**2), axis=1) # agrego features elevados a las potencias que elegí

skf = StratifiedKFold(n_splits=5, shuffle=True) # igual que en las clases tome 5 folds y shuffle=True para que los grupos sean al azar
skf.get_n_splits(X_1, y) # armo los folds a partir de los datos

auc_values = [] # acá es donde van a ir a parar los AUCs de cada fold
scores = np.array([])     # acá es donde van a ir a parar los scores computados para todos los casos

for train_index, test_index in skf.split(X_1, y): # va generando los indices que corresponden a train y test en cada fold
    X_train, X_test = X_1[train_index], X_1[test_index] # arma que es dato de entrenamiento y qué es dato de evaluación
    y_train, y_test = y[train_index], y[test_index]     # idem con los targets

    regLog_1 = LogisticRegression(penalty = 'l2',C=16, max_iter=10000) # defino el modelo
    regLog_1.fit(X_train, y_train) # Ajusto el modelo con los datos de entrenamiento

    probas_test = regLog_1.predict_proba(X_test)  # probabilidades con datos de evaluación
    fpr_test, tpr_test, thresholds_test = roc_curve(y_test, probas_test[:,1]) # para plotear curva ROC con datos de entrenamiento
    auc_test = roc_auc_score(y_test, probas_test[:,1]) #  AUC con datos de evaluación

    auc_values.append(auc_test)
    scores = np.concatenate((scores,probas_test[:,1]),axis=0)

print("Estos son los valores AUC para cada fold:")
print(auc_values)
print("Estos es el promedio de todos los AUC:")
print(np.mean(auc_values))
print("Estos son las probabilidades para cada sample:")
print(scores)

Estos son los valores AUC para cada fold:
[0.6610549943883277, 0.7202797202797202, 0.8053613053613053, 0.7400932400932401, 0.8053613053613055]
Estos es el promedio de todos los AUC:
0.7464301130967798
Estos son las probabilidades para cada sample:
[0.28980062 0.302609   0.69033221 0.27621301 0.78643099 0.75151454
 0.54571231 0.60762669 0.63148028 0.74651771 0.49798619 0.82535245
 0.75069521 0.59111898 0.27536427 0.31013511 0.40791133 0.07926366
 0.46471484 0.92210709 0.61807301 0.82080628 0.80571265 0.59555979
 0.30589513 0.64218071 0.40997438 0.89367796 0.73513962 0.76478211
 0.76980436 0.70347    0.67898973 0.34654345 0.87374457 0.69905272
 0.67898973 0.32297965 0.35622843 0.74651771 0.15180485 0.7684419
 0.87182252 0.64196176 0.17424995 0.49788023 0.38137994 0.57370071
 0.44889568 0.24216261 0.72763039 0.61990641 0.35994748 0.84078859
 0.93555283 0.4814853  0.5642462  0.49199149 0.85142242 0.12969984
 0.62718993 0.74739489 0.56672247 0.6607987  0.43792575 0.47361125
 0.66626861 0.85

In [ ]:
drive.mount('/content/drive')
path = '/content/drive/My Drive/anticoncepcion_indonesia_test.csv'
df_test1 = pd.read_csv(path)
df_test1.columns
print(df_test1)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
      edad_mujer  educacion_mujer  ...  nivel_vida  exposicion_medios
0             30                2  ...           1                  0
1             31                3  ...           3                  0
2             23                1  ...           2                  1
3             46                1  ...           1                  1
4             36                4  ...           4                  0
...          ...              ...  ...         ...                ...
1172          29                3  ...           4                  0
1173          24                3  ...           2                  0
1174          39                1  ...           1                  1
1175          38                1  ...           3                  1
1176          29                2  ...           1                  0

[1177 rows x 9 columns]


Lo único que tuve que tener en cuenta para predecir con los datos que nos dieron, fue que tenáa que tener los features al cuadrado, no solo lineal

In [ ]:
campos = ['edad_mujer','educacion_mujer', 'educacion_hombre', 'numero_hijos', 'mujer_trabaja','nivel_vida']
X_test1 = np.array(df_test1[campos])
X_2 = np.concatenate((X_test1, X_test1**2), axis=1)

In [ ]:
probas = regLog_1.predict_proba(X_2)
print(probas)

[[0.89041692 0.10958308]
 [0.89037402 0.10962598]
 [0.41499933 0.58500067]
 ...
 [0.93561992 0.06438008]
 [0.88004365 0.11995635]
 [0.74878252 0.25121748]]


In [ ]:
import numpy as np
save_path= '/content/drive/My Drive/Victoria_CailletBois.csv'

np.savetxt(save_path, probas, delimiter=",")